In [ ]:
!pip install pymysql
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime as dt
import datetime
from IPython.display import JSON
import sqlalchemy
#import passwords as pw
import pymysql
from IPython.display import JSON
from pytz import timezone


In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_European_cities_by_population_within_city_limits"
response = requests.get(url)
response
#json_response = response.JSON()
#json_response

In [275]:
cities_dict = {
    'city':[],
    'country':[],
    'population':[],
    'lat':[],
    'long':[]
}

In [276]:
soup = BeautifulSoup(response.content)

In [277]:
for i,row in enumerate(soup.select("table.wikitable tr")): 
  if i == 0:
    continue
  cities_dict['city'].append(row.select('td a')[0].get_text())
  cities_dict['country'].append(row.select('td')[2].select_one('a').get_text())
  pop_str = row.select('td')[3].select_one('span').get_text()
  cities_dict['population'].append(int(pop_str.replace(',','')))
  geo = row.select('td')[7].select_one("span.geo").get_text()
  geo = geo.split(";")
  cities_dict['lat'].append(float(geo[0].strip()))
  cities_dict['long'].append(float(geo[1].strip()))

In [278]:
cities=pd.DataFrame(cities_dict)

In [279]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        36 non-null     object 
 1   country     36 non-null     object 
 2   population  36 non-null     int64  
 3   lat         36 non-null     float64
 4   long        36 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.5+ KB


In [280]:
cities.to_csv('cities.csv', index=False)

Connectto to the MySQL database

In [284]:
host = 'wbs-project-db.cp44hp46l3sj.eu-north-1.rds.amazonaws.com'
schema = 'gans_projekt' # The name of your database (schema) might be different
user = 'admin'
password = 'Pasword'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [285]:
cities.to_sql('cities',con=con,if_exists='append',index=False)

36

In [286]:
cities

,city,country,population,lat,long
0,Istanbul,Turkey,15907951,41.013611,28.955000
1,Moscow,Russia,13104177,55.750000,37.616667
2,London,United Kingdom,8799800,51.507222,-0.127500
3,Saint Petersburg,Russia,5600044,59.950000,30.300000
4,Berlin,Germany,3850809,52.516667,13.383333
5,Madrid,Spain,3305408,40.383333,-3.716667
6,Kyiv,Ukraine,2952301,50.450000,30.523333
7,Rome,Italy,2749031,41.900000,12.500000
8,Bucharest,Romania,2162281,44.432500,26.103889
9,Paris,France,2102650,48.856700,2.350800


Whethear

In [221]:
host = 'wbs-project-db.cp44hp46l3sj.eu-north-1.rds.amazonaws.com'
schema = 'gans_projekt' # The name of your database (schema) might be different
user = 'admin'
password = 'Password'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [222]:
cities = pd.read_sql('cities',con=con)
cities

,city_id,city,country,population,lat,long
0,1,Istanbul,Turkey,15907951,41.0136,28.95500
1,2,Moscow,Russia,13104177,55.7500,37.61670
2,3,London,United Kingdom,8799800,51.5072,-0.12750
3,4,Saint Petersburg,Russia,5600044,59.9500,30.30000
4,5,Berlin,Germany,3850809,52.5167,13.38330
5,6,Madrid,Spain,3305408,40.3833,-3.71667
6,7,Kyiv,Ukraine,2952301,50.4500,30.52330
7,8,Rome,Italy,2749031,41.9000,12.50000
8,9,Bucharest,Romania,2162281,44.4325,26.10390
9,10,Paris,France,2102650,48.8567,2.35080


In [223]:
API_key = 'key'

In [224]:
weather_dict = {'city_id':[],
                'time':[],
                'temp':[],
                'wind_speed':[]}


In [225]:
for i, city in enumerate(cities['city_id']):
    url = f"http://api.openweathermap.org/data/2.5/forecast?lat={cities.iloc[i]['lat']}&lon={cities.iloc[i]['long']}&appid={API_key}&units=metric"
    response = requests.get(url)
    weather_json = response.json()
    for time in weather_json['list'][:9]:
        weather_dict['city_id'].append(city)
        weather_dict['time'].append(time['dt_txt'])
        weather_dict['temp'].append(time['main']['temp'])
        weather_dict['wind_speed'].append(time['wind']['speed'])

In [226]:
weather=pd.DataFrame(weather_dict)
weather

,city_id,time,temp,wind_speed
0,1,2023-06-07 06:00:00,18.89,5.72
1,1,2023-06-07 09:00:00,20.19,6.29
2,1,2023-06-07 12:00:00,21.82,6.89
3,1,2023-06-07 15:00:00,21.43,7.30
4,1,2023-06-07 18:00:00,18.67,6.15
...,...,...,...,...
319,36,2023-06-07 18:00:00,19.59,2.45
320,36,2023-06-07 21:00:00,19.35,2.16
321,36,2023-06-08 00:00:00,18.82,4.19
322,36,2023-06-08 03:00:00,18.38,4.45


Tabdil be SQL

In [227]:
weather.to_sql('weather',con=con,if_exists='append',index=False)

324

Afzodan column weather_id

In [228]:
weather = pd.read_sql('weather',con=con)
weather

,weather_id,city_id,time,temp,wind_speed
0,1,1,2023-06-07 06:00:00,18.89,5.72
1,2,1,2023-06-07 09:00:00,20.19,6.29
2,3,1,2023-06-07 12:00:00,21.82,6.89
3,4,1,2023-06-07 15:00:00,21.43,7.30
4,5,1,2023-06-07 18:00:00,18.67,6.15
...,...,...,...,...,...
319,320,36,2023-06-07 18:00:00,19.59,2.45
320,321,36,2023-06-07 21:00:00,19.35,2.16
321,322,36,2023-06-08 00:00:00,18.82,4.19
322,323,36,2023-06-08 03:00:00,18.38,4.45


Airports

In [287]:
host = 'wbs-project-db.cp44hp46l3sj.eu-north-1.rds.amazonaws.com'
schema = 'gans_projekt' # The name of your database (schema) might be different
user = 'admin'
password = 'Password'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [288]:
cities= pd.read_sql('cities',con=con)
cities

,city_id,city,country,population,lat,long
0,1,Istanbul,Turkey,15907951,41.0136,28.95500
1,2,Moscow,Russia,13104177,55.7500,37.61670
2,3,London,United Kingdom,8799800,51.5072,-0.12750
3,4,Saint Petersburg,Russia,5600044,59.9500,30.30000
4,5,Berlin,Germany,3850809,52.5167,13.38330
...,...,...,...,...,...,...
67,68,Cologne,Germany,1073096,50.9364,6.95278
68,69,Voronezh,Russia,1051995,51.6717,39.21060
69,70,Perm,Russia,1027153,58.0000,56.31670
70,71,Volgograd,Russia,1025662,48.7000,44.51670


In [289]:
lat = [52.3112, 53.3300,51.3026,53.2846,41.2258]
lon = [13.2418,10.0000, 0.739,2.1443,2.1037]

list_airports = []

for i in range(len(lat)):
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

    querystring = {"lat":lat[i],"lon":lon[i],"radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}

    headers = {
        "X-RapidAPI-Key": "key",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    list_airports.append(response.json()['items'])

In [242]:
list_airports

[[{'icao': 'EDDB',
   'iata': 'BER',
   'name': 'Berlin, Berlin Brandenburg',
   'shortName': 'Brandenburg',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.35139, 'lon': 13.493889},
   'countryCode': 'DE'}],
 [{'icao': 'EDDH',
   'iata': 'HAM',
   'name': 'Hamburg',
   'shortName': 'Hamburg',
   'municipalityName': 'Hamburg',
   'location': {'lat': 53.6304, 'lon': 9.988229},
   'countryCode': 'DE'}],
 [{'icao': 'EGMC',
   'iata': 'SEN',
   'name': 'Southend',
   'shortName': 'Southend',
   'municipalityName': 'Southend',
   'location': {'lat': 51.5714, 'lon': 0.695555},
   'countryCode': 'GB'}],
 [],
 [{'icao': 'LEBL',
   'iata': 'BCN',
   'name': 'Barcelona',
   'shortName': 'Barcelona',
   'municipalityName': 'Barcelona',
   'location': {'lat': 41.2971, 'lon': 2.078459},
   'countryCode': 'ES'}]]

In [290]:
airports_dict = {'icao':[]}
for item in list_airports:
    if isinstance(item, list) and item: # Check if the item is not empty
         airports_dict['icao'].append(item[0]['icao'])
        #airports_dict['name'].append(item[0]['name'])

In [291]:
airports= pd.DataFrame(airports_dict)


In [245]:
airports

,icao
0,EDDB
1,EDDH
2,EGMC
3,LEBL


In [292]:
airports['city_id'] = [1, 2, 3, 5]

In [247]:
airports

,icao,city_id
0,EDDB,1
1,EDDH,2
2,EGMC,3
3,LEBL,5


In [293]:
airports.to_sql('airports',con=con,if_exists='append',index=False)

4

In [294]:
airports= pd.read_sql('airports',con=con)
airports

,icao,city_id
0,EDDB,1
1,EDDH,2
2,EGMC,3
3,LEBL,5


Flights

In [253]:
host = 'wbs-project-db.cp44hp46l3sj.eu-north-1.rds.amazonaws.com'
schema = 'gans_projekt' # The name of your database (schema) might be different
user = 'admin'
password = 'Password'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [255]:
airports = pd.read_sql('airports',con=con)
airports

,icao,city_id
0,EDDB,1
1,EDDH,2
2,EGMC,3
3,LEBL,5


In [256]:
querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
headers = {
	"X-RapidAPI-Key": "Key",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

In [257]:
from datetime import datetime, date, timedelta

In [258]:
today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
tomorrow = (today + timedelta(days=1))
    
times = [["00:00","11:59"],["12:00","23:59"]]

In [259]:
icao_list = airports['icao'].to_list()

In [260]:
#icao_list[1:2]

['EDDH']

In [266]:
flights_dict = {'ICAO':[],'departure_airport':[],'local_time':[]}
icao_list = airports['icao'].to_list()

for time in times:
    
    for icao in icao_list:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        response = requests.request("GET", url, headers=headers, params=querystring)
        print(response)
        flight_json = response.json()
        
        for flight in flight_json['arrivals']:
            flights_dict['ICAO'].append(icao)
            try:
                flights_dict['departure_airport'].append(flight['departure']['airport']['icao'])
            except:
                flights_dict['departure_airport'].append('unknown')
            try:
                flights_dict['local_time'].append(flight['arrival']['scheduledTimeLocal'])
            except:
                flights_dict['local_time'].append(pd.NaT)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [204]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [267]:
flights = pd.DataFrame(flights_dict)

In [268]:
flights['local_time'] = pd.to_datetime(flights['local_time'])

In [269]:
flights.to_sql('flight',con=con,if_exists='append',index=False)

942

In [270]:
flights_check = pd.read_sql('flight',con=con)
flights_check

,ICAO,departure_airport,local_time
0,EDDH,LGIR,2023-06-08 07:30:00
1,EDDH,EDDF,2023-06-08 07:40:00
2,EDDH,EDDL,2023-06-08 07:50:00
3,EDDH,EHAM,2023-06-08 07:55:00
4,EDDH,LFSB,2023-06-08 08:00:00
...,...,...,...
1110,EDDH,EDDF,2023-06-08 22:35:00
1111,EDDH,LTAI,2023-06-08 22:40:00
1112,EDDH,LEMD,2023-06-08 22:40:00
1113,EDDH,HEGN,2023-06-08 22:45:00
